In [1]:
import SmithWaterman as sw
import Preprocess as fp
import DiGraph as dg
from datetime import datetime

# Preassembly

In [2]:
print("start experiment at", datetime.now())

print("start distance computation at", datetime.now())
processed = fp.preprocess('coverage_5.fasta')
print("distance computation ends at", datetime.now())

start experiment at 2020-07-07 18:19:22.618109
start distance computation at 2020-07-07 18:19:22.620538
distance computation ends at 2020-07-07 18:19:24.233973


# Alignment and Graph Construction

In [3]:
print("alignment/graph construction starts at", datetime.now())

# Declare a graph
G = dg.DiGraph()

# set ploidy level
ploidy_level = 3

#Comparing the sequences
for seq in processed:
    
    for seq2 in processed[seq]: 
        # store seq, score and prefix/suffix info
        scoring, pre_su, i, max_i, j, max_j= sw.smith_waterman(seq, seq2)

        # if it is not a contained read
        if(scoring != 0):
            # if seq2 is a prefix
            if (pre_su): 
                
                G.add_edge(seq2, seq , scoring, j, max_j, i, max_i)
            
            else:  
                
                G.add_edge(seq , seq2, scoring, i, max_i, j, max_j)
        
print("alignment/graph construction ends at", datetime.now())

graph alignment/construction starts at 2020-07-07 18:19:24.254583
overlap score = 7491
overlap score = 23
overlap score = 23
overlap score = 22
overlap score = 131
overlap score = 6762
overlap score = 13389
overlap score = 5338
overlap score = 12164
overlap score = 23
overlap score = 11727
overlap score = 7618
overlap score = 11091
overlap score = 23
overlap score = 23
overlap score = 5780
overlap score = 5754
overlap score = 15
overlap score = 23
overlap score = 23
overlap score = 23
overlap score = 23
overlap score = 23
overlap score = 23
overlap score = 14
overlap score = 23
overlap score = 5058
overlap score = 7544
overlap score = 4225
overlap score = 7544
overlap score = 23
overlap score = 7302
overlap score = 5546
overlap score = 6958
overlap score = 23
overlap score = 23
overlap score = 5243
overlap score = 1405
overlap score = 15
overlap score = 23
overlap score = 23
overlap score = 23
overlap score = 23
overlap score = 5424
overlap score = 15
overlap score = 5424
overlap score

In [4]:
G.num_of_subgraphs()

The number of disconnected components in the graph is  1


# Graph Cleaning

In [5]:
print("start graph cleaning at", datetime.now())
G.remove_cycle()

sorted_node = G.topological_sort()

G.transitive_reduction(sorted_node)

print("graph cleaning ends at", datetime.now())

start graph cleaning at 2020-07-08 20:30:36.611240
All the cycles are removed from the graph.
Transitive reduction is completed.
graph cleaning ends at 2020-07-08 20:30:36.634536


# Graph Reduction

In [6]:
print("start graph reduction at", datetime.now())

G.filter_graph(sorted_node, ploidy_level)

print("graph reduction ends at", datetime.now())

start graph reduction at 2020-07-08 20:30:36.653710
Graph is reduced to best overlap graph.
graph reduction ends at 2020-07-08 20:30:36.659655


# Haplotype Formation

In [7]:
print("start haplotype formation at", datetime.now())

result = G.haplotype(ploidy_level)

for i in result:
    print(i)

print("haplotype formation ends at", datetime.now())

print("experiment ends at", datetime.now())

start haplotype formation at 2020-07-08 20:30:36.681689
['...
haplotype formation ends at 2020-07-08 20:30:36.700952
experiment ends at 2020-07-08 20:30:36.701148


In [8]:
print("first  copy")
gen = fp.fasta_iter("NC_001328.1.copy0.fasta")
for i in gen:
    scoring, pre_su, i, max_i, j, max_j= sw.smith_waterman(result, i)

print("second copy")
gen = fp.fasta_iter("NC_001328.1.copy1.fasta")
for i in gen:
    scoring, pre_su, i, max_i, j, max_j= sw.smith_waterman(result, i)
    
print("third copy")
gen = fp.fasta_iter("NC_001328.1.copy2.fasta")
for i in gen:
    scoring, pre_su, i, max_i, j, max_j= sw.smith_waterman(result, i)

first  copy
overlap score = 0
second copy
overlap score = 0
third copy
overlap score = 0


In [12]:
import Function as ft

ofile = open("seq_cov5.fasta", "w")

for i in range(len(result)):
    
    seq_num = i + 1
    ofile.write(">NC_seq" + int_to_Roman(seq_num) + "\n" +result[i] + "\n")

ofile.close()

In [11]:

def int_to_Roman(num):
    val = [
        1000, 900, 500, 400,
        100, 90, 50, 40,
        10, 9, 5, 4,
        1
        ]
    syb = [
        "M", "CM", "D", "CD",
        "C", "XC", "L", "XL",
        "X", "IX", "V", "IV",
        "I"
        ]
    roman_num = ''
    i = 0
    while  num > 0:
        for _ in range(num // val[i]):
            roman_num += syb[i]
            num -= val[i]
        i += 1
    return roman_num
